In [1]:
import pandas as pd
import seaborn as sns
import notebooks.signature_functions as signature_functions


# Kinases highly active over DSRCT with respect to other Entities

In [4]:
# preprocess
import warnings
warnings.filterwarnings('ignore')
report_directory = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/2023.02.07_CJ_batch82_corrected'
annotation_file = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Searches/patient_annotation_mixed_cohort_230207_w_tall.csv'

kinase_scores_file = f'{report_directory}/kinase_results/kinase_scores.tsv'
kinase_scores_file_annotation = f'{report_directory}/kinase_results/scored_peptides.tsv'


In [5]:
kinase_scores = pd.read_csv(kinase_scores_file,sep='\t')
kinase_scores_annotation_df = pd.read_csv(kinase_scores_file_annotation,sep='\t')
annotation_file = pd.read_csv(annotation_file)
dsrct_patinets = annotation_file['Sample name'][annotation_file['Histologic Subtype'] == 'DSRCT'].tolist()
kinase_scores.index = kinase_scores['PSP Kinases']
kinase_scores_df = kinase_scores.transpose()
all_patinets =  annotation_file['Sample name'].unique().tolist()
kinase_scores_df = kinase_scores_df[kinase_scores_df.index.isin(all_patinets)]
kinase_list = kinase_scores_df.columns
kinase_scores_df['Entity'] = 'others'
kinase_scores_df['Entity'][kinase_scores_df.index.isin(dsrct_patinets)] = ['DSRCT']
t_result = signature_functions.one_vs_all_t_test(kinase_scores_df,kinase_list, 'DSRCT','Entity')
new_sginatures = signature_functions.get_final_signatures_from_t_test_results_by_criteria(t_result,p_value_threshold=0.01,
                                                                                          fdr_threshold=None,delta_threshold=1)
sub = kinase_scores_df[new_sginatures['Gene Names']]
sub.index = kinase_scores_df.index

sub['patients'] = sub.index

In [9]:
kinase_scores_annotation_df = kinase_scores_annotation_df[['PSP Kinases', 'Modified sequence']]
counts_mod_seq_df  = kinase_scores_annotation_df.groupby('PSP Kinases').count()
final_res_df = sub.transpose()
final_res_df = t_result.merge(counts_mod_seq_df,right_index=True, left_on='Gene Names')

In [15]:
final_res_df.to_excel(f'{report_directory}/DSRCT_kinase_activity_vs_others.xlsx')

In [ ]:
protein = sub.columns

sub = pd.melt(sub ,id_vars=['patients'],value_vars=protein,  value_name='kinase_scores')
sub['Entity'] = 'others'
sub['Entity'][sub.patients.isin(dsrct_patinets)] = 'DSRCT'
sns.catplot(
    data=sub, kind="swarm",
    x="PSP Kinases", y="kinase_scores", hue="Entity",
    aspect=5
)
